# Database connection test

# Preparation

In [1]:
# Imports
import sqlite3
import pandas as pd

## Main connection

In [2]:
# Connect to main database
db_path = '../TsakonianDB.sqlite3'
conn = sqlite3.connect(db_path)

## Auxiliary functions

In [3]:
def query(query: str,
          cursor = conn.cursor()):

      cursor.execute(query)
      fetched = cursor.fetchall()

      if len(fetched) > 0:
            temp_df = pd.DataFrame(fetched, columns=[i[0] for i in cursor.description])
            return temp_df
      else:
            print('Query executed successfully. No results to show.')

# Add words from main dictionary to the database

## Load dictionary

In [22]:
# Load main dictionary
main_df_path = '../data/tables/main.xlsx'
main_df = pd.read_excel(main_df_path)
main_df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,tsakonian,greek,paradigm,source_id,F1,F2,F3
1,NaN,FILTER,FILTER,FILTER,FILTER,FILTER,FILTER,FILTER
2,NaN,άβατ̇ε,άκλαυτος,Ε,1,άβατ̇ε,άβατ̇ε-άκλαυτος,άβατ̇ε-άκλαυτος-Ε
3,NaN,άγιε,εκκλησία,Α4,3,άγιε,άγιε-εκκλησία,άγιε-εκκλησία-Α4
4,NaN,άγο,άλογο,Α0,1,άγο,άγο-άλογο,άγο-άλογο-Α0
...,...,...,...,...,...,...,...,...
1069,NaN,κρόπο,κοπριά,Θ0,3,κρόπο,κρόπο-κοπριά,κρόπο-κοπριά-Θ0
1070,NaN,καμάρα,καμάρα,Θ,3,καμάρα,καμάρα-καμάρα,καμάρα-καμάρα-Θ
1071,NaN,κοπέα,κοπέλα,Θ,3,κοπέα,κοπέα-κοπέλα,κοπέα-κοπέλα-Θ
1072,NaN,προβάτα,προβατίνα,Θ,3,προβάτα,προβάτα-προβατίνα,προβάτα-προβατίνα-Θ


In [23]:
### Format main dictionary ###
if 'tsakonian' not in main_df.columns:      
    # Extract column headers
    main_df_headers = main_df.iloc[0, 1:5].tolist()

    # Extract entries
    main_df_entries = main_df.iloc[2:, 1:5]

    # Add column headers
    main_df_entries.columns = main_df_headers

    # Replace main_df
    main_df = main_df_entries.copy()

main_df

,tsakonian,greek,paradigm,source_id
2,άβατ̇ε,άκλαυτος,Ε,1
3,άγιε,εκκλησία,Α4,3
4,άγο,άλογο,Α0,1
5,άγουστε,αύγουστος,NaN,1
6,άζ̌α,"αραία, όχι συχνά",NaN,1
...,...,...,...,...
1069,κρόπο,κοπριά,Θ0,3
1070,καμάρα,καμάρα,Θ,3
1071,κοπέα,κοπέλα,Θ,3
1072,προβάτα,προβατίνα,Θ,3


## Manage duplicates

In [18]:
# Remove duplicates with the exact same information
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = main_df.drop_duplicates()
print(f'Entries after removing duplicates: {len(main_df)}')

Entries before removing duplicates: 1071
Entries after removing duplicates: 1071


In [24]:
# Show duplicates
main_df[main_df.duplicated(subset = 'tsakonian', keep = False)].sort_values(by = 'tsakonian')

,tsakonian,greek,paradigm,source_id
7,άθρωπο,άνθροπος,Α6,1
1018,άθρωπο,"άνδρας, σύζυγος",Α0,3
25,άτσ̌ωπο,άνδρας,Α1,1
1017,άτσ̌ωπο,άνθροπος,Α0,3
27,έατε,έλατο,A6,1
...,...,...,...,...
897,χαμεούκ̇ου,χαμηλώνω,Ρ,1
975,όζακα,μυρμήγκι των δέντρων,Α3,3
926,όζακα,είδος μυρμηγκιού,Α3,1
973,όντα,δόντι,Α3,3


In [26]:
# Keep the Tsakonian-Greek duplicate with the lowest ID
drop_duplicates = False

if drop_duplicates:
    print(f'Entries before removing duplicates: {len(main_df)}')
    main_df = (main_df
            .sort_index()
            .drop_duplicates(subset = ['tsakonian', 'greek'], keep = 'first')
            .sort_values(by = 'tsakonian')
            .reset_index(drop = True)
            )
    print(f'Entries after removing duplicates: {len(main_df)}')
    main_df

## Save to main Excel

In [21]:
# Save main_df back to the Excel file
save_changes = False

if save_changes:
    main_df.to_excel(main_df_path, index = False)
    print('Changes consolidated.')

Changes consolidated.


# Upload main_df to the database

In [27]:
# Show the dataframe
main_df

,tsakonian,greek,paradigm,source_id
2,άβατ̇ε,άκλαυτος,Ε,1
3,άγιε,εκκλησία,Α4,3
4,άγο,άλογο,Α0,1
5,άγουστε,αύγουστος,NaN,1
6,άζ̌α,"αραία, όχι συχνά",NaN,1
...,...,...,...,...
1069,κρόπο,κοπριά,Θ0,3
1070,καμάρα,καμάρα,Θ,3
1071,κοπέα,κοπέλα,Θ,3
1072,προβάτα,προβατίνα,Θ,3


In [21]:
main_df.to_sql('dictionary_entry', 
               conn, 
               if_exists='replace', 
               index=False,
               dtype = {'tsakonian': 'varchar(50)',
                        'greek': 'varchar(200)',
                        'paradigm': 'varchar(5)',
                        'source_id': 'bigint'
               })

1071

# Update sources table

In [ ]:
# Read sources.xlsx
sources_df_path = '../data/tables/sources.xlsx'
sources_df = pd.read_excel(sources_df_path)
sources_df.head()

In [ ]:
# Copy the sources into the database
sources_df.to_sql('dictionary_source', 
                  conn, 
                  if_exists='replace', 
                  index=False,
                  dtype = {'id': 'bigint',
                           'source': 'varchar(100)',
                           'author': 'varchar(100)',
                           'year': 'varchar(10)',
                           'notes': 'varchar(100)'
                  })

In [ ]:
# Copy the full sources table into the database
sources_df.to_sql('dictionary_source', 
                  conn, 
                  if_exists='replace', 
                  index=False,
                  dtype = {'id': 'bigint',
                           'source': 'varchar(100)',
                           'author': 'varchar(100)',
                           'year': 'varchar(10)',
                           'notes': 'varchar(100)'
                  })

# Delete the temporary table if it exists
try:
    query("DROP TABLE sqlitestudio_temp_table;")
except:
    pass

# Add primary keys and foreign keys by recreating the table
# Obtained from SQLiteStudio
recreating_query = """PRAGMA foreign_keys = 0;

CREATE TABLE sqlitestudio_temp_table AS SELECT *
                                            FROM dictionary_source;

DROP TABLE dictionary_source;

CREATE TABLE dictionary_source (
    source_id   INTEGER       PRIMARY KEY AUTOINCREMENT,
    title       VARCHAR (300),
    url         VARCHAR (200)
);

INSERT INTO dictionary_source (
                                    source_id,
                                    title,
                                    url
                                )
                                SELECT source_id,
                                         title,
                                         url
                                    FROM sqlitestudio_temp_table;

DROP TABLE sqlitestudio_temp_table;

PRAGMA foreign_keys = 1;"""

# Execute queries in a loop
# Only one query can be executed at a time
for q in recreating_query.split(';'):
      query(q)

# Save changes
conn.commit()